# Absctraction Scorer

In [17]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [26]:
import pandas as pd

df = pd.read_csv("./voice_classified.csv")
df.voice = df.voice.apply(eval)
df.clauses_text_final = df.clauses_text_final.apply(eval)
df

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx
0,2546,36,48,Sometimes I wish that,my mom would come home more often.,[my mom would come home more often],[A_pron_x],0
1,2507,1,90,My family,has a cat...I don't have a cat though I have 2...,"[has a cat, I don t, have a cat, though I have...","[A_def, A_def, A_def, A_def]",1
2,2510,33,33,When I am nervous,my hands get shaky,[my hands get shaky],[A_pron_x],2
3,2509,22,43,At times I worry about,"my own dreams, nightmares I have thousands of ...","[my own dreams nightmares, I have thousands of...","[Undefined, A_def]",3
4,3356,34,47,Technology,its crazy and i wish i could keep up,"[and i wish, i could keep up]","[A_def, A_def]",4
5,2537,26,26,When I get mad,I sometimes use big voice then I go somewhere ...,"[I sometimes use big voice, then I go somewher...","[A_def, P_get_x, A_def]",5
6,1869,25,25,My main problem is,a feeling that I am not entitled,"[a feeling, that I am not entitled]","[Undefined, P_bevb_x]",6
7,3127,29,29,If my mother,"was a different lady, I would not be who I am.","[was a different lady, I would not be, who I am]","[P_bevb_x, A_def, P_bevb_x]",7
8,1832,26,26,When I get mad,"it takes a long time to build up, but I am slo...","[it takes a long time but, to build up, I am s...","[A_def, A_def, P_bevb_x, A_def]",8
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","[I felt safe, that I could receive a warm welc...","[P_get_x, A_pron_x]",9


In [27]:
from nltk.corpus import wordnet as wn, stopwords
ignore_words = list(set(stopwords.words('english')))
ignore_words = ignore_words + ['keep']

DEBUG_ABSTRACTION_HIERARCHY = False
class GenericTree:
    def __init__(self, nltk_word, print_pad = ""):
        self.word = nltk_word
        self.print_pad = print_pad

    def get_max_depth(self):
        if len(self.children) == 0:
            return 0
        return max([tree.get_max_depth() for tree in self.children ])+1
    
    def print_tree(self):
        print("{}>{}".format(self.print_pad, self.word.lemma_names()[0]))
        for tree in self.children:
            tree.print_tree()
            
class HypernymTree(GenericTree):
    def __init__(self, nltk_word, pos, print_pad = ""):
        super().__init__(nltk_word, print_pad)
        if len(nltk_word.hypernyms()) == 0:
            self.children = [] 
        else:
            hyper = nltk_word.hypernyms() 
            hyper = [x for x in hyper if x.pos() == pos]
            self.children = [HypernymTree(x, pos, print_pad = "{}==".format(self.print_pad)) for x in hyper]
        
class HyponymTree(GenericTree):
    def __init__(self, nltk_word, pos, print_pad = ""): 
        super().__init__(nltk_word, print_pad)
        if len(nltk_word.hyponyms()) == 0:
            self.children = [] 
        else:
            hypo = nltk_word.hyponyms()
            hypo = [x for x in hypo if x.pos() == pos]
            self.children = [HyponymTree(x, pos, print_pad = "{}==".format(self.print_pad)) for x in hypo]


In [30]:
from nltk.tokenize import word_tokenize

def hypernym_hierarchy(toks, pos):
    hyper_trees = [HypernymTree(tok, pos) for tok in toks]
    hypers = [tree.get_max_depth() for tree in hyper_trees]
    hypo_trees = [HyponymTree(tok, pos) for tok in toks]
    hypos = [tree.get_max_depth() for tree in hypo_trees]
    if DEBUG_ABSTRACTION_HIERARCHY:
        for tree in hyper_trees:
            print("#####################################################3")
            tree.print_tree()
        for tree in hypo_trees:
            print("#####################################################3")
            tree.print_tree()

    assert len(hypers) == len(hypos)
    op = [hypos[i]*1./(hypers[i] + hypos[i]) if hypers[i]>0 or hypos[i]>0 else 0 for i in range(len(hypers))]
    #print(op)
    return max(op)

def hypernym_score(word):
    toks = wn.synsets(word)
    if len(toks) == 0 or word in ignore_words:
        return 0

    op = hypernym_hierarchy(toks, toks[0].pos())
    return round(op, 2)
    
def score_abstraction(clauses):
    op = []
    for clause in clauses:
        op.append(max([hypernym_score(x) for x in clause]))
    return op

df['abstraction_score'] = df.clauses_text_final.apply(score_abstraction)
df.sample(frac=1).head(20)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score
46,2338,1,1,Raising a family,"Captivates my whole being, just as every windo...","[Captivates my whole being, just as every wind...","[A_pron_x, A_def, P_bevb_x]",46,"[0.25, 0.14, 0.14]"
34,2215,13,40,We could make the world a better place if,"the mystery was okay to be as a question, that...","[the mystery was okay to be as a question, tha...","[P_bevb_x, A_def, P_bevb_x, A_def, P_bevb_x, A...",34,"[0.14, 0.14, 0.14, 0.25, 0.25, 0.25, 0.12, 0.1..."
7,3127,29,29,If my mother,"was a different lady, I would not be who I am.","[was a different lady, I would not be, who I am]","[P_bevb_x, A_def, P_bevb_x]",7,"[0.14, 0.22, 0.22]"
14,1529,10,10,When people are helpless,At times I try to find other ways of doing thi...,"[At times, I try to find other ways of, doing ...","[Undefined, A_def, A_def, A_def, A_def, A_def,...",14,"[0.11, 0.22, 0.14, 0.22, 0.12]"
8,1832,26,26,When I get mad,"it takes a long time to build up, but I am slo...","[it takes a long time but, to build up, I am s...","[A_def, A_def, P_bevb_x, A_def]",8,"[0.14, 0.12, 0.22]"
3,2509,22,43,At times I worry about,"my own dreams, nightmares I have thousands of ...","[my own dreams nightmares, I have thousands of...","[Undefined, A_def]",3,"[0.14, 0.22]"
26,3099,22,43,At times I worry about,"big, big existential questions, and little mic...",[big big existential questions and little micr...,[Undefined],26,[0.25]
40,1814,25,25,My main problem is,how my narccissim gets in the way of seeing th...,"[how my narccissim gets in the way of, seeing ...","[A_pron_x, A_def]",40,"[0.25, 0.14]"
2,2510,33,33,When I am nervous,my hands get shaky,[my hands get shaky],[A_pron_x],2,[0.14]
31,2690,28,83,A teacher has the right to,I don't mean to be oppositional. . but this se...,"[but, I don t mean to be oppositional, this se...","[Undefined, P_get_x, P_get_x, P_bevb_x, A_def,...",31,"[0.12, 0.22, 0.25, 0.25, 0.25, 0.22, 0.22, 0.2..."


In [31]:
df.to_csv("./abstraction_scored.csv", index = False)